In [1]:
!pip install tensorflow==2.4.1

     |████████████████████████████████| 394.3 MB 10 kB/s s eta 0:00:01ta 0:00:01
     |████████████████████████████████| 42 kB 1.6 MB/s  eta 0:00:01
     |████████████████████████████████| 3.8 MB 29.4 MB/s eta 0:00:01
     |████████████████████████████████| 14.8 MB 55.4 MB/s eta 0:00:01
     |████████████████████████████████| 462 kB 45.3 MB/s eta 0:00:01
     |████████████████████████████████| 6.0 MB 43.8 MB/s eta 0:00:01
     |████████████████████████████████| 781 kB 57.1 MB/s eta 0:00:01
     |████████████████████████████████| 4.9 MB 43.2 MB/s eta 0:00:01
  Attempting uninstall: numpy
    Found existing installation: numpy 1.18.5
    Uninstalling numpy-1.18.5:
      Successfully uninstalled numpy-1.18.5
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.36.1
    Uninstalling grpcio-1.36.1:
      Successfully uninstalled grpcio-1.36.1
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.1.0
    Uninstalling tensorfl

In [2]:
import tensorflow as tf
from tensorflow.keras.layers import Conv3D, ZeroPadding3D
from tensorflow.keras.layers import MaxPooling3D
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten
from tensorflow.keras.layers import Bidirectional, TimeDistributed
from tensorflow.keras.layers import GRU
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Lambda
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam

In [19]:
def ctc_lambda_func(args):
    y_pred, labels, input_length, label_length = args
    y_pred = y_pred[:, :, :]
    return K.ctc_batch_cost(labels, y_pred, input_length, label_length)

def CTC(name, args):
    return Lambda(ctc_lambda_func, output_shape=(1,), name=name)(args)

In [20]:
class LipNet(object):
    def __init__(self, img_c=3, img_w=100, img_h=50, frames_n=75, absolute_max_string_len=32, output_size=28):
        self.img_c = img_c
        self.img_w = img_w
        self.img_h = img_h
        self.frames_n = frames_n
        self.absolute_max_string_len = absolute_max_string_len
        self.output_size = output_size
        self.build()

    def build(self):
        if K.image_data_format() == 'channels_first':
            input_shape = (self.img_c, self.frames_n, self.img_w, self.img_h)
        else:
            input_shape = (self.frames_n, self.img_w, self.img_h, self.img_c)

        self.input_data = Input(name='the_input', shape=input_shape, dtype='float32')

        self.zero1 = ZeroPadding3D(padding=(1, 2, 2), name='zero1')(self.input_data)
        self.conv1 = Conv3D(32, (3, 5, 5), strides=(1, 2, 2), activation='relu', kernel_initializer='he_normal', name='conv1')(self.zero1)
        self.maxp1 = MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2), name='max1')(self.conv1)
        self.drop1 = Dropout(0.5)(self.maxp1)

        self.zero2 = ZeroPadding3D(padding=(1, 2, 2), name='zero2')(self.drop1)
        self.conv2 = Conv3D(64, (3, 5, 5), strides=(1, 1, 1), activation='relu', kernel_initializer='he_normal', name='conv2')(self.zero2)
        self.maxp2 = MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2), name='max2')(self.conv2)
        self.drop2 = Dropout(0.5)(self.maxp2)

        self.zero3 = ZeroPadding3D(padding=(1, 1, 1), name='zero3')(self.drop2)
        self.conv3 = Conv3D(96, (3, 3, 3), strides=(1, 1, 1), activation='relu', kernel_initializer='he_normal', name='conv3')(self.zero3)
        self.maxp3 = MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2), name='max3')(self.conv3)
        self.drop3 = Dropout(0.5)(self.maxp3)

        self.resh1 = TimeDistributed(Flatten())(self.drop3)

        self.gru_1 = Bidirectional(GRU(256, return_sequences=True, kernel_initializer='Orthogonal', name='gru1'), merge_mode='concat')(self.resh1)
        self.gru_2 = Bidirectional(GRU(256, return_sequences=True, kernel_initializer='Orthogonal', name='gru2'), merge_mode='concat')(self.gru_1)

        # transforms RNN output to character activations:
        self.dense1 = Dense(self.output_size, kernel_initializer='he_normal', name='dense1')(self.gru_2)

        self.y_pred = Activation('softmax', name='softmax')(self.dense1)

        self.labels = Input(name='the_labels', shape=[self.absolute_max_string_len], dtype='float32')
        self.input_length = Input(name='input_length', shape=[1], dtype='int64')
        self.label_length = Input(name='label_length', shape=[1], dtype='int64')

        self.loss_out = CTC('ctc', [self.y_pred, self.labels, self.input_length, self.label_length])

        self.model = Model(inputs=[self.input_data, self.labels, self.input_length, self.label_length], outputs=self.loss_out)
    
    def summary(self):
        Model(inputs=self.input_data, outputs=self.y_pred).summary()

In [21]:
lipnet = LipNet(3,128,128,75,75,32)

In [22]:
lipnet.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
the_input (InputLayer)       [(None, 75, 128, 128, 3)] 0         
_________________________________________________________________
zero1 (ZeroPadding3D)        (None, 77, 132, 132, 3)   0         
_________________________________________________________________
conv1 (Conv3D)               (None, 75, 64, 64, 32)    7232      
_________________________________________________________________
max1 (MaxPooling3D)          (None, 75, 32, 32, 32)    0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 75, 32, 32, 32)    0         
_________________________________________________________________
zero2 (ZeroPadding3D)        (None, 77, 36, 36, 32)    0         
_________________________________________________________________
conv2 (Conv3D)               (None, 75, 32, 32, 64)    1536

In [23]:
lipnet.model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
the_input (InputLayer)          [(None, 75, 128, 128 0                                            
__________________________________________________________________________________________________
zero1 (ZeroPadding3D)           (None, 77, 132, 132, 0           the_input[0][0]                  
__________________________________________________________________________________________________
conv1 (Conv3D)                  (None, 75, 64, 64, 3 7232        zero1[0][0]                      
__________________________________________________________________________________________________
max1 (MaxPooling3D)             (None, 75, 32, 32, 3 0           conv1[0][0]                      
____________________________________________________________________________________________

In [24]:
import numpy as np
x = np.zeros((2 ,75 ,128,128 , 3))
labels = np.zeros((2,75))
input_length = np.ones((2,1))*75
label_length = np.ones((2,1))*150

y = np.zeros((2,1))

In [25]:
adam = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)

lipnet.model.compile(loss={'ctc': lambda y_true, y_pred: y_pred}, optimizer=adam)

In [26]:
lipnet.model.fit([x,labels,input_length,label_length],y,epochs = 2)

Epoch 1/2
1/1 [==============================] - 20s 20s/step - loss: inf
Epoch 2/2
1/1 [==============================] - 13s 13s/step - loss: inf
